In [ ]:
## Trainset과 Validation set 나누기 
import os
import shutil
import random
import pandas as pd

# 원본 데이터 경로
data_dir = r'E:\musinsa\data_file\cropped_images_no_detection_copy'

# 훈련 및 검증 데이터셋 경로
train_dir = r'E:\musinsa\data_file\train_product'
val_dir = r'E:\musinsa\data_file\val_product'

# 검증 데이터 비율 설정 (예: 20% 검증, 80% 훈련)
val_split = 0.2

# CSV 파일 경로
csv_file_path = r'E:\musinsa\image_상품전체사진수.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 이미지 수가 12개 이상인 폴더만 필터링
filtered_folders = df[df['ImageCount'] >= 12]['FolderName'].apply(str).tolist()
print(f"필터링된 폴더: {filtered_folders}")

# train과 val 디렉토리가 없으면 생성
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 클래스별로 이미지 나누기
for class_name in os.listdir(data_dir):
    print(f"현재 처리 중인 클래스: {class_name}")
    if class_name in filtered_folders:
        class_path = os.path.join(data_dir, class_name)

        if os.path.isdir(class_path):
            # 이미지 파일 목록 가져오기
            images = os.listdir(class_path)
            print(f"이미지 파일 목록: {images}")
            if not images:
                print(f"{class_name}에 이미지가 없습니다.")
                continue

            random.shuffle(images)  # 이미지를 랜덤하게 섞음

            # 검증 데이터와 훈련 데이터로 나누기
            split_idx = int(len(images) * val_split)
            val_images = images[:split_idx]
            train_images = images[split_idx:]

            # 클래스별 훈련/검증 디렉토리 생성
            train_class_dir = os.path.join(train_dir, class_name)
            val_class_dir = os.path.join(val_dir, class_name)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(val_class_dir, exist_ok=True)

            # 훈련 데이터 복사
            for image in train_images:
                src = os.path.join(class_path, image)
                dst = os.path.join(train_class_dir, image)
                print(f"훈련 데이터 복사: {src} -> {dst}")
                shutil.copy(src, dst)

            # 검증 데이터 복사
            for image in val_images:
                src = os.path.join(class_path, image)
                dst = os.path.join(val_class_dir, image)
                print(f"검증 데이터 복사: {src} -> {dst}")
                shutil.copy(src, dst)

print("훈련 및 검증 데이터셋 분할 완료.")


In [ ]:
### 상품 1000개 모델 학습 전  -증강 
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import cv2
import numpy as np
import random
from rembg import remove, new_session
import numpy as np
import cv2
import random
from PIL import Image

# GPU가 사용 가능한지 확인하고 메모리 성장을 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"사용 가능한 GPU: {gpus}")
    except RuntimeError as e:
        print(f"GPU 설정 중 오류 발생: {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU로 실행합니다.")

# 데이터셋 경로 설정
train_dir = r'E:\musinsa\data_file\train'
val_dir = r'E:\musinsa\data_file\val'

# 이미지 크기와 배치 크기 설정
img_height, img_width = 224, 224
batch_size = 32
session = new_session(model_name='u2net_human_seg')

# 가우시안 블러 함수 정의
def add_gaussian_blur(image):
    image = np.array(image, dtype=np.float32)  # 이미지를 numpy 배열로 변환
    if random.random() < 0.5:  # 50% 확률로 블러 적용
        image = cv2.GaussianBlur(image, (5, 5), 0)  # 가우시안 블러 적용
    return image
# 배경 제거 함수 정의
def remove_background(image):
    # 일정 확률로 배경 제거 적용 (예: 50%)
    if random.random() < 0.5:  # 50% 확률로 배경 제거 적용
        pil_image = Image.fromarray((image * 255).astype(np.uint8))  # NumPy 배열을 PIL 이미지로 변환
        pil_image = remove(pil_image, session=session)  # rembg를 사용해 배경 제거
        # RGBA(4채널)를 RGB(3채널)로 변환
        pil_image = pil_image.convert("RGB")
        return np.array(pil_image).astype(np.float32) / 255.0  # PIL 이미지를 NumPy 배열로 다시 변환하고 정규화
    else:
        return image  # 배경 제거 없이 원본 이미지 반환

# 데이터 증강 및 전처리 설정 (증강된 이미지 생성)
train_datagen = ImageDataGenerator(
    preprocessing_function=lambda img: remove_background(add_gaussian_blur(img)),  # 배경 제거 후 가우시안 블러 적용
    rotation_range=40,                # 0~40도 범위에서 임의의 회전
    width_shift_range=0.2,            # 수평 방향으로 최대 40% 임의 이동
    height_shift_range=0.2,           # 수직 방향으로 최대 20% 임의 이동
    shear_range=0.2,                  # 전단 변환 강도
    zoom_range=0.2,                   # 확대/축소 범위 (최대 20%)
    horizontal_flip=False,            # 수평 반전
    vertical_flip=False,              # 수직 반전
    fill_mode='nearest',              # 회전이나 이동으로 인해 생긴 빈 픽셀을 채우는 방법 ('nearest', 'constant', 'reflect', 'wrap')
    brightness_range=[0.7, 1.2],      # 밝기 조정 (최소 70%, 최대 120%)
    channel_shift_range=0.2,          # 색상 채널 이동 범위
    rescale=1./255,                   # 픽셀 값을 0-1 범위로 정규화
    dtype='float32'                   # 출력 데이터의 dtype 설정
)
# 원본 이미지에서 증강된 이미지 생성 후 총 1000개로 맞추기
augmented_dir = os.path.join(train_dir, 'augmented')
os.makedirs(augmented_dir, exist_ok=True)

target_num_images = 200  # 각 클래스당 총 이미지 개수 (기존 이미지 + 증강 이미지)

for class_dir in os.listdir(train_dir):
    if not os.path.isdir(os.path.join(train_dir, class_dir)):
        continue
    
    source_dir = os.path.join(train_dir, class_dir)
    target_dir = os.path.join(augmented_dir, class_dir)
    os.makedirs(target_dir, exist_ok=True)
    
    current_images = os.listdir(source_dir)
    num_existing_images = len(current_images)
    num_needed_images = target_num_images - num_existing_images
    
    if num_needed_images <= 0:
        # 이미 1000개 이상의 이미지가 있는 경우
        for filename in current_images:
            img_path = os.path.join(source_dir, filename)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
            img.save(os.path.join(target_dir, filename))  # 이미지를 복사
        continue
    
    # 필요한 증강 횟수를 계산
    num_images_per_original = num_needed_images // num_existing_images
    remainder = num_needed_images % num_existing_images

    # 각 이미지에 대해 고르게 증강을 적용
    for i, filename in enumerate(current_images):
        img_path = os.path.join(source_dir, filename)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
        x = tf.keras.preprocessing.image.img_to_array(img)
        x = x.reshape((1,) + x.shape)
        
        # 원본 이미지 복사
        tf.keras.preprocessing.image.save_img(os.path.join(target_dir, filename), x[0])
        
        # 증강된 이미지 생성 (각 이미지를 고르게 증강)
        for j in range(num_images_per_original):
            for batch in train_datagen.flow(x, batch_size=1, save_to_dir=target_dir, save_prefix='aug', save_format='jpeg'):
                break
        
        # 나머지 증강 작업을 일부 이미지에 추가로 적용
        if i < remainder:
            for batch in train_datagen.flow(x, batch_size=1, save_to_dir=target_dir, save_prefix='aug', save_format='jpeg'):
                break

# 원본 이미지와 증강된 이미지 모두를 포함한 훈련 데이터 로드
train_datagen = ImageDataGenerator(
    rescale=1./255  # 정규화
)

train_ds = train_datagen.flow_from_directory(
    augmented_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# 검증 데이터 로드
val_datagen = ImageDataGenerator(rescale=1./255)

val_ds = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)




In [ ]:
##상품 1000개 학습  
import tensorflow as tf

# GPU가 사용 가능한지 확인하고 메모리 성장을 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"사용 가능한 GPU: {gpus}")
    except RuntimeError as e:
        print(f"GPU 설정 중 오류 발생: {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU로 실행합니다.")

# ResNet50 모델 로드 (사전 학습된 ImageNet 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# ResNet50 모델의 모든 층을 학습 가능하게 설정
for layer in base_model.layers:
    layer.trainable = False

# Sequential 모델 구성
model = Sequential()

# ResNet50 기본 모델  
model.add(base_model)


# 완전 연결층(Fully Conneced Layers) 
model.add(Flatten())  # 특징 맵을 1차원으로 변환 
model.add(Dense(1024, activation='relu'))  # Fully Connected 층 추가
model.add(Dropout(0.5))  # 과적합 방지를 위한 Dropout 층 추가 

# 출력층(Output Layer)
model.add(Dense(train_ds.num_classes, activation='softmax'))

# 모델 컴파일
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# 체크포인트 콜백 설정
checkpoint_cb = ModelCheckpoint(r'E:\musinsa\checkpoints\resnet50_custom_model_checkpoint_epoch_{epoch:02d}.h5', 
                                save_best_only=True,
                                monitor='accuracy', 
                                mode='max',
                                verbose=1)

# 모델 학습
history = model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=100,
    validation_data=val_ds,
    callbacks=[checkpoint_cb]
)

# 학습 완료 후 모델 저장 (최종 상태)
model.save('resnet50_custom_model.h5')

In [ ]:
### 상품 500개  학습  
import tensorflow as tf
# GPU가 사용 가능한지 확인하고 메모리 성장을 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"사용 가능한 GPU: {gpus}")
    except RuntimeError as e:
        print(f"GPU 설정 중 오류 발생: {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU로 실행합니다.")


# 데이터셋 경로 설정
train_dir = r'E:\musinsa\data_file\train_product'
val_dir = r'E:\musinsa\data_file\val_product'

# 이미지 크기와 배치 크기 설정
img_height, img_width = 224, 224
batch_size = 32

# 데이터 증강 및 전처리 설정
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    brightness_range=[0.7, 1.2],
    channel_shift_range=0.2,
    rescale=1./255,
    dtype='float32'
)

# 검증 데이터는 증강하지 않음
val_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터 로드
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# 검증 데이터 로드
val_ds = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# 증강된 이미지 확인
x_batch, y_batch = next(train_ds)
for i in range(5):
    plt.figure(figsize=(2, 2))
    plt.imshow(x_batch[i])
    plt.axis('off')
    plt.show()

# ResNet50 모델 로드 (사전 학습된 ImageNet 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# ResNet50 모델의 모든 층을 학습 불가능하게 설정
for layer in base_model.layers:
    layer.trainable = False       #False 설정하면 모든 레이어가 고정(freeze)

# Sequential 모델 구성
model = Sequential()

# ResNet50 기본 모델 추가
model.add(base_model)

# 추가적인 CNN 계층 추가
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

# 분류기 계층 추가
model.add(Flatten())
model.add(Dense(1024, activation='relu'))

# 출력층: 클래스 수를 자동으로 설정
model.add(Dense(train_ds.num_classes, activation='softmax'))

# 모델 컴파일
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])


# 체크포인트 콜백 설정
checkpoint_cb = ModelCheckpoint(r'E:\musinsa\checkpoints\resnet50_custom_model_V3_checkpoint_epoch_{epoch:02d}.h5', 
                                save_best_only=True,
                                monitor='accuracy', 
                                mode='max',
                                verbose=1)

# 모델 학습
history = model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=20,
    validation_data=val_ds,
    callbacks=[checkpoint_cb]
)

# 학습 완료 후 모델 저장 (최종 상태)
model.save('resnet50_custom_model_500class.h5')

In [ ]:
#추가학습  500class 
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"사용 가능한 GPU: {gpus}")
    except RuntimeError as e:
        print(f"GPU 설정 중 오류 발생: {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU로 실행합니다.")


# ResNet50 모델의 모든 층을 학습 불가능하게 설정
for layer in model.layers:
    layer.trainable = False       #False 설정하면 모든 레이어가 고정(freeze)
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])
# 체크포인트 콜백 설정 (계속해서 체크포인트 저장)
checkpoint_cb = ModelCheckpoint(r'E:\musinsa\checkpoints\resnet50_custom_model_500class_checkpoint_epoch_{epoch:02d}.h5', 
                                save_best_only=True,
                                monitor='accuracy', 
                                mode='max',
                                verbose=1)

# 데이터셋 경로 설정
train_dir = r'E:\musinsa\data_file\train_product'
val_dir = r'E:\musinsa\data_file\val_product'

# 이미지 크기와 배치 크기 설정
img_height, img_width = 224, 224
batch_size = 32

# 데이터 증강 및 전처리 설정
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    brightness_range=[0.7, 1.2],
    channel_shift_range=0.2,
    rescale=1./255,
    dtype='float32'
)

# 검증 데이터는 증강하지 않음
val_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 데이터 로드
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# 검증 데이터 로드
val_ds = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
# 추가 학습 수행
history = model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=100,  # 남은 에폭만큼 학습 (예: 7번째 에폭까지 했으므로, 8~20 에폭 추가)
    initial_epoch=7,  # 이전 학습이 종료된 에폭 번호를 지정
    validation_data=val_ds,
    callbacks=[checkpoint_cb]
)

# 학습 완료 후 모델 저장 (최종 상태)
model.save('resnet50_custom_model_500class_final.h5')



In [ ]:
# 학습시킨 모델 불러오기 
import tensorflow as tf

model = tf.keras.models.load_model(r'E:\musinsa\checkpoints\resnet50_custom_model_V3_checkpoint_epoch_05.h5')
model.get_config() #기본값 확인 


In [ ]:
###softmax로 반환된 확률 테스트 -상위 몇개 할지 (상품 1000개 모델 )

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# GPU가 사용 가능한지 확인하고 메모리 성장을 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"사용 가능한 GPU: {gpus}")
    except RuntimeError as e:
        print(f"GPU 설정 중 오류 발생: {e}")
else:
    print("GPU를 찾을 수 없습니다. CPU로 실행합니다.")

# 경로 설정
data_dir = r'E:\musinsa\data_file\train\augmented'
val_dir = r'E:\musinsa\data_file\test'

# 폴더 이름(클래스 이름) 가져오기 및 정렬
class_names = sorted(os.listdir(data_dir))

# 클래스 이름과 라벨 매핑 확인
print(f'Class names: {class_names}')

# 모델 로드 (이미 학습된 모델이 있다고 가정)
model = tf.keras.models.load_model(r'E:\musinsa\checkpoints\resnet50_custom_model_checkpoint_epoch_18.h5')

# 정확도 계산을 위한 변수 초기화
total_images = 0
top_1_correct = 0
top_5_correct = 0
top_10_correct = 0
top_15_correct = 0
top_20_correct = 0
top_30_correct = 0

# val_10 디렉토리의 모든 하위 폴더를 순회하며 이미지 처리
for class_folder in os.listdir(val_dir):
    folder_path = os.path.join(val_dir, class_folder)
    if not os.path.isdir(folder_path):
        continue
    
    print(f"\nProcessing images in folder: {class_folder}")

    for img_file in os.listdir(folder_path):
        total_images += 1
        image_path = os.path.join(folder_path, img_file)
        
        # 이미지 로드 및 전처리
        img = image.load_img(image_path, target_size=(224, 224))  # 이미지 크기 모델에 맞게 조정
        img_array = image.img_to_array(img)  # 이미지를 배열로 변환
        img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
        img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  # ResNet50 전처리
        
        # 예측
        predictions = model.predict(img_array)
        
        # 상위 5개의 클래스 인덱스와 확률 추출
        top_5_indices = np.argsort(predictions[0])[::-1][:5]
        top_5_probabilities = predictions[0][top_5_indices]
        top_5_class_names = [class_names[i] for i in top_5_indices]
        # 상위 10개의 클래스 인덱스와 확률 추출
        top_10_indices = np.argsort(predictions[0])[::-1][:10]
        top_10_probabilities = predictions[0][top_10_indices]
        top_10_class_names = [class_names[i] for i in top_10_indices]

        # 상위 15개의 클래스 인덱스와 확률 추출
        top_15_indices = np.argsort(predictions[0])[::-1][:15]
        top_15_probabilities = predictions[0][top_15_indices]
        top_15_class_names = [class_names[i] for i in top_15_indices]

        # 상위 20개의 클래스 인덱스와 확률 추출
        top_20_indices = np.argsort(predictions[0])[::-1][:20]
        top_20_probabilities = predictions[0][top_20_indices]
        top_20_class_names = [class_names[i] for i in top_20_indices]   
        # 상위 30개의 클래스 인덱스와 확률 추출
        top_30_indices = np.argsort(predictions[0])[::-1][:20]
        top_30_probabilities = predictions[0][top_30_indices]
        top_30_class_names = [class_names[i] for i in top_30_indices]       

        # 결과 출력
        print(f'Image: {img_file}')
        for i in range(5):
            print(f"Top {i+1} prediction: {top_5_class_names[i]} (Probability: {top_5_probabilities[i]:.4f})")
        
        # 정확도 계산
        if class_folder == top_5_class_names[0]:
            top_1_correct += 1
        if class_folder in top_5_class_names:
            top_5_correct += 1
        if class_folder in top_10_class_names:
            top_10_correct += 1
        if class_folder in top_15_class_names:
            top_15_correct += 1
        if class_folder in top_20_class_names:
            top_20_correct += 1     
        if class_folder in top_30_class_names:
            top_20_correct += 1     
        # 실제 폴더 이름과 상위 예측 클래스 비교
        match = class_folder in top_5_class_names
        print(f'Match with folder name in top 5 predictions: {match}')  # True if folder name is in top 5 predictions
# 최종 정확도 계산
top_1_accuracy = top_1_correct / total_images
top_5_accuracy = top_5_correct / total_images
top_10_accuracy = top_10_correct / total_images
top_15_accuracy = top_15_correct / total_images
top_20_accuracy = top_20_correct / total_images
top_30_accuracy = top_30_correct / total_images


print(f"\nTotal images: {total_images}")
print(f"Top-1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top_5_accuracy:.4f}")
print(f"Top-10 Accuracy: {top_10_accuracy:.4f}")
print(f"Top-15Accuracy: {top_15_accuracy:.4f}")
print(f"Top-20 Accuracy: {top_20_accuracy:.4f}")
print(f"Top-30 Accuracy: {top_30_accuracy:.4f}")




In [ ]:
###최종이다 -소형 모델  ##


import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# GPU가 사용 가능한지 확인하고 메모리 성장을 설정
# 경로 설정
data_dir = r'E:\musinsa\data_file\train\augmented'
val_dir = r'E:\musinsa\data_file\test'

# 폴더 이름(클래스 이름) 가져오기 및 정렬
class_names = sorted(os.listdir(data_dir))

# 클래스 이름과 라벨 매핑 확인
print(f'Class names: {class_names}')

# 모델 로드 (이미 학습된 모델이 있다고 가정)
model = tf.keras.models.load_model(r'E:\musinsa\checkpoints\resnet50_custom_model_V3_checkpoint_epoch_05.h5')

# 정확도 계산을 위한 변수 초기화
total_images = 0
top_1_correct = 0
top_5_correct = 0


# val_10 디렉토리의 모든 하위 폴더를 순회하며 이미지 처리
for class_folder in os.listdir(val_dir):
    folder_path = os.path.join(val_dir, class_folder)
    if not os.path.isdir(folder_path):
        continue
    
    print(f"\nProcessing images in folder: {class_folder}")

    for img_file in os.listdir(folder_path):
        total_images += 1
        image_path = os.path.join(folder_path, img_file)
        
        # 이미지 로드 및 전처리
        img = image.load_img(image_path, target_size=(224, 224))  # 이미지 크기 모델에 맞게 조정
        img_array = image.img_to_array(img)  # 이미지를 배열로 변환
        img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가
        img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  # ResNet50 전처리
        
        # 예측
        predictions = model.predict(img_array)
        
        # 상위 5개의 클래스 인덱스와 확률 추출
        top_5_indices = np.argsort(predictions[0])[::-1][:5]
        top_5_probabilities = predictions[0][top_5_indices]
        top_5_class_names = [class_names[i] for i in top_5_indices]    
        # 결과 출력
        print(f'Image: {img_file}')
        for i in range(5):
            print(f"Top {i+1} prediction: {top_5_class_names[i]} (Probability: {top_5_probabilities[i]:.4f})")
        
        # 정확도 계산
        if class_folder == top_5_class_names[0]:
            top_1_correct += 1
        if class_folder in top_5_class_names:
            top_5_correct += 1
        # 실제 폴더 이름과 상위 예측 클래스 비교
        match = class_folder in top_5_class_names
        print(f'Match with folder name in top 5 predictions: {match}')  # True if folder name is in top 5 predictions
# 최종 정확도 계산
top_1_accuracy = top_1_correct / total_images
top_5_accuracy = top_5_correct / total_images


print(f"\nTotal images: {total_images}")
print(f"Top-1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top-5 Accuracy: {top_5_accuracy:.4f}")
